## 1. Configuration

In [1]:
# Configuration
NUM_AGENTS = 3
STREAM_NAME = "agent_stream"
REDIS_HOST = "localhost"
REDIS_PORT = 6379
RUN_DURATION_SECONDS = 60  # How long to run the demo

initial_prompt_template = (
    "You are agent {agent_id} in a multi-agent social network debate about '{topic}'. " \
    "You strongly believe in the following statement: {unique_prompt} " \
    "Defend this statement in your responses. Prove your point is correct compared to others." \
    "Respond thoughtfully and concisely, keeping the conversation going."
)

converstation_starter = (
    "Let's begin our discussion about {topic}. " \
    "What are your initial thoughts on this subject?"
)


print(f"Configuration: {NUM_AGENTS} agents, stream='{STREAM_NAME}', duration={RUN_DURATION_SECONDS}s")

Configuration: 3 agents, stream='agent_stream', duration=60s


## 2. Imports

In [2]:
import asyncio
import nest_asyncio

# Enable nested event loops for Jupyter
nest_asyncio.apply()

from agents.network_agent import NetworkAgent
from agents.prompt_configs.generate_prompt import PromptGenerator
from controller.time_manager import TimeManager
from controller.order_manager import OrderManager
from network.cache import RedisCache
from logs.logger import Logger, console_logger

print("Imports successful!")

Imports successful!


## 3. Initialize Shared Components

In [10]:
# Initialize PromptGenerator with shared topic
prompt_generator = PromptGenerator()
topic = prompt_generator.get_topic()
print(f"Shared discussion topic: {topic}")

# Generate unique prompts for each agent
agent_prompts = prompt_generator.generate_multiple_prompts(NUM_AGENTS)
for i, prompt in enumerate(agent_prompts):
    print(f"Agent {i+1} prompt: {prompt[:150]}...")

Shared discussion topic: sports and fitness
Agent 1 prompt: Considering stability, sports and fitness appears dull; predict is required....
Agent 2 prompt: In terms of reliability, sports and fitness should be seen as controversial and we should criticize accordingly....
Agent 3 prompt: My experience as a teacher shows that sports and fitness is progressive and impacts berries availability; even a penguin can notice the change....


In [11]:
# Initialize TimeManager with 3-second global interval
time_manager = TimeManager(global_interval=3.0)
print(f"TimeManager initialized with {time_manager.global_interval}s global interval")

# Initialize RedisCache for storing agent message histories
message_cache = RedisCache(host=REDIS_HOST, port=REDIS_PORT)
print(f"RedisCache connected to {REDIS_HOST}:{REDIS_PORT}")

# Initialize Logger for recording agent publishes
logger = Logger(num_agents=NUM_AGENTS)
print(f"Logger initialized, writing to: {logger.file_path}")

TimeManager initialized with 3.0s global interval
RedisCache connected to localhost:6379
Logger initialized, writing to: /home/sammli/llm-network/logs/network_logs/log_20260112-005712_3.log


## 4. Create Agents

In [12]:
# Create agents with unique prompts but shared topic
agents = []
for i in range(NUM_AGENTS):
    agent_id = f"agent_{i+1}"
    init_prompt = initial_prompt_template.format(
        agent_id=agent_id,
        topic=topic,
        unique_prompt=agent_prompts[i]
    )
    agent = NetworkAgent(
        id=agent_id,
        init_prompt=init_prompt,
        stream_name=STREAM_NAME,
        stream_group=f"group_{i+1}",
        redis_host=REDIS_HOST,
        redis_port=REDIS_PORT,
        time_manager=time_manager,
        order_manager=None,  # Will set after OrderManager is created
        message_cache=message_cache,
        logger=logger,
    )
    agents.append(agent)
    print(f"Created {agent_id}")

print(f"\nTotal agents created: {len(agents)}")

Created agent_1
Created agent_2
Created agent_3

Total agents created: 3


In [13]:
# Initialize OrderManager with all agents
order_manager = OrderManager(
    agents=agents,
    message_cache=message_cache,
    redis_host=REDIS_HOST,
    redis_port=REDIS_PORT,
)
print("OrderManager initialized")

# Inject OrderManager into each agent
for agent in agents:
    agent.order_manager = order_manager
print("OrderManager injected into all agents")

OrderManager initialized
OrderManager injected into all agents


## 5. Start Agents and Run Conversation

In [14]:
async def start_agents():
    """Start all agents."""
    print("Starting agents...")
    for agent in agents:
        await agent.start()
    print(f"All {NUM_AGENTS} agents are now running!")

# Start agents (Jupyter supports top-level await)
await start_agents()

[INFO] 2026-01-12 00:57:26,888 - Consumer group 'group_1' already exists for stream 'agent_stream'.
[INFO] 2026-01-12 00:57:26,889 - Agent agent_1 is consuming from stream 'agent_stream' as part of group 'group_1'.
[INFO] 2026-01-12 00:57:26,890 - Agent agent_1 ignored its own message 1768200315569-0.
[INFO] 2026-01-12 00:57:26,890 - Agent agent_1 started and is listening for messages.
[INFO] 2026-01-12 00:57:26,892 - Consumer group 'group_2' already exists for stream 'agent_stream'.
[INFO] 2026-01-12 00:57:26,892 - Agent agent_2 is consuming from stream 'agent_stream' as part of group 'group_2'.


Starting agents...


[INFO] 2026-01-12 00:57:27,979 - Agent agent_2 started and is listening for messages.
[INFO] 2026-01-12 00:57:27,982 - Consumer group 'group_3' already exists for stream 'agent_stream'.
[INFO] 2026-01-12 00:57:27,983 - Agent agent_3 is consuming from stream 'agent_stream' as part of group 'group_3'.
[INFO] 2026-01-12 00:57:28,992 - Agent agent_3 started and is listening for messages.


All 3 agents are now running!


[INFO] 2026-01-12 00:57:31,013 - Agent agent_1 skipped (designated responder is None).


In [ ]:
async def kick_off_conversation():
    """Start the conversation with an initial message."""
    # Use the conversation starter template (the init prompt template requires agent_id/unique_prompt)
    initial_message = converstation_starter.format(topic=topic)
    print(f"Agent 1 starting conversation: {initial_message}")
    await agents[0].publish_message(initial_message)
    print("Initial message published!")

# Kick off conversation
await kick_off_conversation()

KeyError: 'agent_id'

In [ ]:
async def run_conversation(duration: int):
    """Let the conversation run for the specified duration."""
    print(f"Running conversation for {duration} seconds...")
    await asyncio.sleep(duration)
    print("Conversation complete!")

# Run the conversation
await run_conversation(RUN_DURATION_SECONDS)

## 6. Cleanup

In [ ]:
async def cleanup():
    """Clean up resources."""
    print("Shutting down...")
    # Stop background consumer tasks so reruns don’t duplicate consumers
    for agent in agents:
        try:
            await agent.stop()
        except Exception as e:
            print(f"Warning: failed stopping {agent.id}: {e}")
    await logger.async_stop()
    await message_cache.close()
    print("Done!")

# Cleanup
await cleanup()

## 7. Inspect Results (Optional)

After running the conversation, you can inspect the logged messages and cached data.

In [ ]:
# Read the log file
with open(logger.file_path, 'r') as f:
    logs = f.read()
print("=== Agent Publish Logs ===")
print(logs)

In [ ]:
async def inspect_cache():
    """Inspect cached messages for each agent."""
    # Reconnect cache for inspection
    cache = RedisCache(host=REDIS_HOST, port=REDIS_PORT)
    print("=== Cached Messages per Agent ===")
    for agent in agents:
        messages = await cache.get_responses(agent.id, last_n=5)
        print(f"\n{agent.id} (last 5 messages):")
        for msg in messages:
            print(f"  - {msg}")
    await cache.close()

asyncio.get_event_loop().run_until_complete(inspect_cache())